# Препроцессинг данных для обучения модели

In [7]:
import os
import shutil
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import cv2

In [2]:
DATASET_PATH = './dataset/'  
NEW_DATASET_PATH = './dataset_merged'

SPLITS = ['train', 'valid', 'test']

# Объединим данные после аугментации

In [3]:
for split in SPLITS:
    # Пути к старым данным
    img_dir = Path(DATASET_PATH) / split / 'images'
    label_dir = Path(DATASET_PATH) / split / 'labels'
    aug_img_dir = Path(DATASET_PATH) / split / 'images_augmented'
    aug_label_dir = Path(DATASET_PATH) / split / 'labels_augmented'

    # Пути к новым данным
    new_img_dir = Path(NEW_DATASET_PATH) / split / 'images'
    new_label_dir = Path(NEW_DATASET_PATH) / split / 'labels'

    os.makedirs(new_img_dir, exist_ok=True)
    os.makedirs(new_label_dir, exist_ok=True)

    # Копируем оригиналы
    for file in tqdm(os.listdir(img_dir), desc=f"Copying images {split}"):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            shutil.copy(str(img_dir / file), str(new_img_dir / file))

    for file in tqdm(os.listdir(label_dir), desc=f"Copying labels {split}"):
        if file.endswith('.txt'):
            shutil.copy(str(label_dir / file), str(new_label_dir / file))

    # Копируем аугментированные
    if os.path.exists(aug_img_dir):
        for file in tqdm(os.listdir(aug_img_dir), desc=f"Copying augmented images {split}"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                shutil.copy(str(aug_img_dir / file), str(new_img_dir / file))

    if os.path.exists(aug_label_dir):
        for file in tqdm(os.listdir(aug_label_dir), desc=f"Copying augmented labels {split}"):
            if file.endswith('.txt'):
                shutil.copy(str(aug_label_dir / file), str(new_label_dir / file))

Copying images train:   0%|          | 0/128 [00:00<?, ?it/s]

Copying labels train:   0%|          | 0/128 [00:00<?, ?it/s]

Copying augmented images train:   0%|          | 0/620 [00:00<?, ?it/s]

Copying augmented labels train:   0%|          | 0/620 [00:00<?, ?it/s]

Copying images valid:   0%|          | 0/25 [00:00<?, ?it/s]

...

Copying augmented labels valid:   0%|          | 0/120 [00:00<?, ?it/s]

Copying images test:   0%|          | 0/36 [00:00<?, ?it/s]

Copying labels test:   0%|          | 0/36 [00:00<?, ?it/s]

Copying augmented images test:   0%|          | 0/171 [00:00<?, ?it/s]

Copying augmented labels test:   0%|          | 0/171 [00:00<?, ?it/s]

In [4]:
old_dataset_path = DATASET_PATH
new_dataset_path = NEW_DATASET_PATH

if os.path.exists(old_dataset_path):
    shutil.rmtree(old_dataset_path)

if os.path.exists(new_dataset_path):
    shutil.move(new_dataset_path, old_dataset_path)
    print("переименовали новую директорию")
else:
    print("что-то не так")

переименовали новую директорию


# Объединение классов

In [5]:
for split in ['train', 'valid', 'test']:
    label_dir = Path(DATASET_PATH) / split / 'labels'

    def process_label_folder(label_path):
        if not os.path.exists(label_path):
            print(f"Папка не найдена: {label_path}")
            return

        for file in tqdm(os.listdir(label_path), desc=f"Processing {label_path.name}"):
            if file.endswith('.txt'):
                file_path = label_path / file

                with open(file_path, 'r') as f:
                    lines = f.readlines()

                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) < 1:
                        continue

                    new_line = '0 ' + ' '.join(parts[1:])
                    new_lines.append(new_line + '\n')

                with open(file_path, 'w') as f:
                    f.writelines(new_lines)

    process_label_folder(label_dir)

Processing labels:   0%|          | 0/748 [00:00<?, ?it/s]

Processing labels:   0%|          | 0/145 [00:00<?, ?it/s]

Processing labels:   0%|          | 0/207 [00:00<?, ?it/s]

# Создание бинарных масок

In [8]:
def load_yolo_segmentation(file_path, img_shape):
    h, w = img_shape[:2]
    with open(file_path, 'r') as f:
        lines = f.readlines()

    polygons = []
    for line in lines:
        data = list(map(float, line.strip().split()))
        coords = np.array(data[1:]).reshape(-1, 2)
        coords[:, 0] *= w  # x
        coords[:, 1] *= h  # y
        polygons.append(coords.astype(np.int32))

    return polygons

In [ ]:
for split in ['train', 'valid', 'test']:
    images_dir = Path(DATASET_PATH) / split / 'images'
    labels_dir = Path(DATASET_PATH) / split / 'labels'
    masks_dir = Path(DATASET_PATH) / split / 'binary_masks'
    
    os.makedirs(masks_dir, exist_ok=True)

    print(f"\nОбработка {split}...")

    for img_file in tqdm(os.listdir(images_dir)):
        if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        base_name = img_file.rsplit('.', 1)[0]
        label_file = labels_dir / f"{base_name}.txt"
        image_path = images_dir / img_file

        if not os.path.exists(label_file):
            print(f"Нет аннотации для {img_file}")
            continue

        image = cv2.imread(str(image_path))
        h, w = image.shape[:2]

        polygons = load_yolo_segmentation(label_file, (h, w))

        mask = np.zeros((h, w), dtype=np.uint8)

        for polygon in polygons:
            cv2.fillPoly(mask, [polygon], color=255)

        mask_file = masks_dir / f"{base_name}.png"
        cv2.imwrite(str(mask_file), mask)

print("\nГотово")


Обработка train...


  0%|          | 0/748 [00:00<?, ?it/s]


Обработка valid...


  0%|          | 0/145 [00:00<?, ?it/s]


Обработка test...


  0%|          | 0/207 [00:00<?, ?it/s]


Готово
